# Atividade 1 - Etapa 2 - Identificar os períodos de deglutição nos arquivos válidos

In [21]:
import pandas as pd

df = pd.read_csv('arquivos.csv')
df.set_index('Nome',inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
Nome,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,22.297441,24.186403,19.642017,21.590790,22.246239,19.611900,22.830761,23.824404,18.229954,22.329424
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,33.231340,31.175938,29.877090,32.193497,28.894571,25.434364,37.405285,56.316820,66.772320,74.781690
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,22.355438,20.060133,20.182419,19.959610,23.543373,22.039034,22.282020,21.076845,25.755106,16.517843
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,31.750853,33.178425,29.853620,26.377653,25.703300,24.194206,25.185480,19.914272,26.392110,23.255760
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,26.043133,22.162785,20.825216,21.485440,20.339930,19.954765,20.697723,23.707054,16.932465,16.452013


## Classificador de Audios Válidos

Classificação geral dos arquivos de audio:

In [23]:
tutor = pd.read_csv('classificacao_anomalias.csv')
tutor = tutor[['File Name','Label']]
tutor['Label'] = tutor['Label'].apply(lambda x: 1 if x == 0 else 0)
tutor.set_index('File Name',inplace=True)
# identificando labels e quantidades
display(tutor['Label'].value_counts())
tutor.head()

Label
0    266
1    224
Name: count, dtype: int64

,Label
File Name,
a00001.mp3,1
a00002.mp3,0
a00003.mp3,1
a00004.mp3,1
a00005.mp3,1


In [24]:
# conectando a tabela de parâmetros da etapa 1 com os sons válidos do tutor
ds = df.join(tutor)
# removo as linhas que não tem identificação de válido
ds.dropna(inplace=True, axis=0)
display(ds.shape)
ds.head()

(490, 112)

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,Label
Nome,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,24.186403,19.642017,21.590790,22.246239,19.611900,22.830761,23.824404,18.229954,22.329424,1
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,31.175938,29.877090,32.193497,28.894571,25.434364,37.405285,56.316820,66.772320,74.781690,0
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,20.060133,20.182419,19.959610,23.543373,22.039034,22.282020,21.076845,25.755106,16.517843,1
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,33.178425,29.853620,26.377653,25.703300,24.194206,25.185480,19.914272,26.392110,23.255760,1
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,22.162785,20.825216,21.485440,20.339930,19.954765,20.697723,23.707054,16.932465,16.452013,1


Utilizando metodologias de aprendizado supervisionado classificador com validação cruzada (CV) para avaliação das técnicas com metrica F1:

In [25]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

X = ds.drop(columns=['Label']).values
y = ds['Label'].values

classificadores = [
    DecisionTreeClassifier(random_state=42,max_depth=5),
    ExtraTreeClassifier(random_state=42,max_depth=5),
    RandomForestClassifier(random_state=42,max_depth=5),
    ExtraTreesClassifier(random_state=42,max_depth=5),
    GradientBoostingClassifier(random_state=42,max_depth=5),
    AdaBoostClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42,max_depth=5),
    LogisticRegression(random_state=42),
]

resultados = []
for cls in tqdm(classificadores):
    res = cross_validate(cls, X, y, cv=5, scoring='f1')
    resultados.append(
        {'metodo': cls.__class__.__name__, 
         'f1': res['test_score'].mean(), 
         'tempo': res['fit_time'].mean(),
         }
    )

warnings.filterwarnings('default')
df_res = pd.DataFrame(resultados)
df_res.sort_values('f1', ascending=False)

100%|██████████| 8/8 [00:18<00:00,  2.25s/it]


,metodo,f1,tempo
7,LogisticRegression,0.575298,0.044220
4,GradientBoostingClassifier,0.556985,2.581231
0,DecisionTreeClassifier,0.542093,0.027940
6,HistGradientBoostingClassifier,0.531629,0.197833
5,AdaBoostClassifier,0.528551,0.349795
2,RandomForestClassifier,0.519410,0.262690
3,ExtraTreesClassifier,0.470698,0.097964
1,ExtraTreeClassifier,0.437442,0.001258


In [86]:
clf = RandomForestClassifier(random_state=42,max_depth=5)
clf.fit(X, y)

Xr = df[df.columns[1:]].values
y_pred = clf.predict(Xr)
df['Valido'] = y_pred
#ds.reset_index(inplace=True)
np.unique(y_pred, return_counts=True)

(array([0., 1.]), array([189, 301], dtype=int64))

In [87]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

blocos = []
for i, row in df.head(50).iterrows():
    nome = row["0"].split(".")[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Inválido' if row['Valido'] == 0 else 'Válido')
        display(Audio(f'dataset/MP3/{row["0"]}'))
    blocos.append(out)
widgets.HBox(blocos)